# **Data Collection**

## Objectives of this notebook

* Fetch data from Kaggle using a json API and prepare it for future processing


### Steps 
* Import packages
* Set working directory
* Fetch data from Kaggle and prepare it for further processing.
* Clean data
* Split data into different environments

## Inputs

* Kaggle JSON file - authentication token 

## Outputs

* Generate Dataset: inputs/datasets/cherryleaves_dataset/cherry-leaves

## Additional Comments

* These steps will allow us to fecth data, clean it, divide it into the different environments in preparation for the machine learning activities. 



----

# Import packages

In [1]:
! pip install -r /workspaces/Mildew_Detection_pjkt/requirements.txt

  Using cached numpy-1.19.2-cp38-cp38-manylinux2010_x86_64.whl (14.5 MB)
  Using cached pandas-1.1.2-cp38-cp38-manylinux1_x86_64.whl (10.4 MB)
  Using cached streamlit-0.85.0-py2.py3-none-any.whl (7.9 MB)
ERROR: Ignored the following versions that require a different python version: 0.55.2 Requires-Python <3.5; 1.25.0 Requires-Python >=3.9; 1.25.0rc1 Requires-Python >=3.9; 1.25.1 Requires-Python >=3.9; 1.25.2 Requires-Python >=3.9; 1.26.0 Requires-Python <3.13,>=3.9; 1.26.0b1 Requires-Python <3.13,>=3.9; 1.26.0rc1 Requires-Python <3.13,>=3.9; 1.26.1 Requires-Python <3.13,>=3.9; 1.26.2 Requires-Python >=3.9; 1.26.3 Requires-Python >=3.9; 2.1.0 Requires-Python >=3.9; 2.1.0rc0 Requires-Python >=3.9; 2.1.1 Requires-Python >=3.9; 2.1.2 Requires-Python >=3.9; 2.1.3 Requires-Python >=3.9; 2.1.4 Requires-Python >=3.9; 2.2.0rc0 Requires-Python >=3.9
ERROR: Could not find a version that satisfies the requirement python-3.8.16 (from versions: none)
ERROR: No matching distribution found for python

In [2]:
import numpy
import os

## Change working directory

**Change the working directory from its current folder to its parent folder**

* Note: I need to change the working directory when running the notebook in the editor, since the notebooks are in a subfolder.

In [3]:
current_dir = os.getcwd()
current_dir

'/workspaces/Mildew_Detection_pjkt/jupyter_notebooks'

**Make the parent of the current directory the new current directory**

In [4]:
os.chdir('/workspaces/Mildew_Detection_pjkt')
print("You set a new current directory")

You set a new current directory


**Confirm the new current directory**

In [5]:
current_dir = os.getcwd()
current_dir

'/workspaces/Mildew_Detection_pjkt'

In [6]:
# Install Kaggle

In [7]:
! pip install kaggle==1.5.12

In [8]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


**Set the kaggle configuration directory to the current working directory**

In [9]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

In [10]:
KaggleDatasetPath = "codeinstitute/cherry-leaves"
DestinationFolder = "inputs/cherryleaves_dataset/cherry-leaves"   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

 95%|███████████████████████████████████▉  | 52.0M/55.0M [00:00<00:00, 78.7MB/s]
100%|██████████████████████████████████████| 55.0M/55.0M [00:00<00:00, 80.6MB/s]


**Unzip the dowloaded file and delete the zip file**

In [11]:
import zipfile
with zipfile.ZipFile(DestinationFolder + '/cherry-leaves.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/cherry-leaves.zip')

---

# Data Preparation

## Data Cleaning

**Check and remove non images files as they will not be used for model training**

In [12]:
def remove_non_image_file(my_data_dir):
    image_extension = ('.png', '.jpg', '.jpeg')
    folders = os.listdir(my_data_dir)

    for folder in folders:
        folder_path = os.path.join(my_data_dir, folder)
        files = os.listdir(folder_path)

        i = []
        j = []
        for given_file in files:
            file_location = os.path.join(folder_path, given_file)

            if os.path.isfile(file_location):
                if not given_file.lower().endswith(image_extension):
                    os.remove(file_location)  # remove non-image file
                    i.append(1)
                else:
                    j.append(1)

        print(f"Folder: {folder_path} - has image file {len(j)}")
        print(f"Folder: {folder_path} - has non-image file {len(i)}")
        

In [13]:
remove_non_image_file(my_data_dir='inputs/cherryleaves_dataset/cherry-leaves')

Folder: inputs/cherryleaves_dataset/cherry-leaves/powdery_mildew - has image file 2104
Folder: inputs/cherryleaves_dataset/cherry-leaves/powdery_mildew - has non-image file 0
Folder: inputs/cherryleaves_dataset/cherry-leaves/cherry-leaves - has image file 0
Folder: inputs/cherryleaves_dataset/cherry-leaves/cherry-leaves - has non-image file 0
Folder: inputs/cherryleaves_dataset/cherry-leaves/healthy - has image file 2104
Folder: inputs/cherryleaves_dataset/cherry-leaves/healthy - has non-image file 0
Folder: inputs/cherryleaves_dataset/cherry-leaves/test - has image file 0
Folder: inputs/cherryleaves_dataset/cherry-leaves/test - has non-image file 0
Folder: inputs/cherryleaves_dataset/cherry-leaves/validation - has image file 0
Folder: inputs/cherryleaves_dataset/cherry-leaves/validation - has non-image file 0
Folder: inputs/cherryleaves_dataset/cherry-leaves/train - has image file 0
Folder: inputs/cherryleaves_dataset/cherry-leaves/train - has non-image file 0


---

In [14]:
## Split train validation test set

In [15]:
import os
import shutil
import random
import joblib


def split_train_validation_test_images(my_data_dir, train_set_ratio, validation_set_ratio, test_set_ratio):

    if train_set_ratio + validation_set_ratio + test_set_ratio != 1.0:
        print("train_set_ratio + validation_set_ratio + test_set_ratio should sum to 1.0")
        return

    # gets classes labels
    labels = os.listdir(my_data_dir)  # it should get only the folder name
    if 'test' in labels:
        pass
    else:
        # create train, test folders with classes labels sub-folder
        for folder in ['train', 'validation', 'test']:
            for label in labels:
                os.makedirs(name=my_data_dir + '/' + folder + '/' + label)

        for label in labels:

            files = os.listdir(my_data_dir + '/' + label)
            random.shuffle(files)

            train_set_files_qty = int(len(files) * train_set_ratio)
            validation_set_files_qty = int(len(files) * validation_set_ratio)

            count = 1
            for file_name in files:
                if count <= train_set_files_qty:
                    # move a given file to the train set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/train/' + label + '/' + file_name)

                elif count <= (train_set_files_qty + validation_set_files_qty):
                    # move a given file to the validation set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/validation/' + label + '/' + file_name)

                else:
                    # move given file to test set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/test/' + label + '/' + file_name)

                count += 1

            os.rmdir(my_data_dir + '/' + label)

I will split the data following convention rules below:
* The training set is divided into a 0.70 ratio of data.
* The validation set is divided into a 0.10 ratio of data.
* The test set is divided into a 0.20 ratio of data.

In [16]:
split_train_validation_test_images(my_data_dir=f"inputs/cherryleaves_dataset/cherry-leaves",
                                   train_set_ratio=0.7,
                                   validation_set_ratio=0.1,
                                   test_set_ratio=0.2
                                   )

---